In [ ]:
from google.colab import files
import pandas as pd
import io

upload_files = files.upload()

file_name = next(iter(upload_files))

df = pd.read_csv(io.StringIO(upload_files[file_name].decode('utf-8')), header=1, delimiter=";")

df

In [ ]:
df = df.drop('ID', axis = 1)

In [ ]:
print(df["EDUCATION"].unique())

In [ ]:
# Combine EDUCATION values 0, 5 and 6 into 4
# This was done because documentation says that the column can only take 4 values (1 = graduate school; 2 = university; 3 = high school; 4 = others).
df["EDUCATION"] = df["EDUCATION"].replace([0, 5, 6], 4)
df["EDUCATION"].unique()

In [ ]:
print(df["MARRIAGE"].unique())

In [ ]:
# Combining MARRIAGE column "0" values to 3
# The documentation says the column can only take 3 values: (1 = married; 2 = single; 3 = others). 0 is neither of those.
df["MARRIAGE"] = df["MARRIAGE"].replace(0, 3)
df["MARRIAGE"].unique()

In [ ]:
# Encoding EDUCATION column:
new_data = pd.get_dummies(df, columns=['EDUCATION', 'MARRIAGE'], drop_first=True)
new_data

In [ ]:
# Encoding SEX column:
new_data['SEX'] = new_data['SEX'] - 1
print(new_data["SEX"].unique())
# 0 = male, 1 = female

In [ ]:
# Normalize continuous values:
from sklearn.preprocessing import StandardScaler

cont_columns = ["AGE", "LIMIT_BAL", "BILL_AMT1", "BILL_AMT2", "BILL_AMT3",
                "BILL_AMT4", "BILL_AMT5", "BILL_AMT6",
                "PAY_AMT1", "PAY_AMT2", "PAY_AMT3", "PAY_AMT4", "PAY_AMT5", "PAY_AMT6"]

scaler = StandardScaler()
new_data[cont_columns] = scaler.fit_transform(new_data[cont_columns])
new_data

In [ ]:
from sklearn.model_selection import train_test_split

x = new_data.drop(columns=["default payment next month"])
y = df["default payment next month"]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=1000)
model.fit(x_train, y_train)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

y_pred = model.predict(x_test)

acc = accuracy_score(y_test, y_pred)
print("Accuracy:", acc)

print(classification_report(y_test, y_pred))

print(confusion_matrix(y_test, y_pred))
